In [11]:
from roboflow import Roboflow

rf = Roboflow(api_key="GFoDeF6xcsj0qTPFO5r7")
project = rf.workspace().project("signa-sync")
model = project.version(3).model

# infer on a local image
#print(model.predict("./vives_6.mp4").json())


# save an image annotated with your predictions
#model.predict("./vives_6.mp4").save("prediction.jpg")

loading Roboflow workspace...
loading Roboflow project...


In [12]:
import cv2

# Opens the Video file
cap= cv2.VideoCapture('vives_5.mp4')
i=0
sampling_rate = 15
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    if i % sampling_rate == 0: # this is the line I added to make it only save one frame every 20
        # frame redim
        frame = cv2.resize(frame, (640, 480))
        cv2.imwrite('./tmp/frame_'+str(i)+'.jpg',frame)
    i+=1

cap.release()

In [4]:
import os
from threading import Thread

class PredictFrame(Thread):
    def __init__(self, frame_path):
        Thread.__init__(self)
        self.frame_path = frame_path
        self.result = None

    def run(self):
        prediction = model.predict(self.frame_path).json()
        prediction = prediction['predictions'][0]['predictions'][0]
        if prediction['confidence'] < 0.2:
            self.result = None
        else:
            self.result = prediction['class']

class FramesPredictor(Thread):
    def __init__(self):
        Thread.__init__(self)
        self.result = None

    def run(self):
        classes_frequency = {}
        threads = []
        for image in os.listdir('./tmp'):
            thread = PredictFrame('./tmp/' + image)
            thread.start()
            threads.append(thread)
        while len(threads) > 0:
            thread = threads.pop()
            thread.join()
            prediction = thread.result
            if prediction in classes_frequency:
                classes_frequency[prediction] += 1
            else:
                classes_frequency[prediction] = 1
        self.result = classes_frequency
frames_predictor = FramesPredictor()
frames_predictor.start()
frames_predictor.join()
print(frames_predictor.result)

{'como_estas': 15, 'vives': 1}


In [13]:

import cv2
import shutil

def start_live_demo():
    # Abre la cámara predeterminada (puedes cambiar a otro número si tienes múltiples cámaras)
    cap = cv2.VideoCapture(1)
    
    frame_number = 0
    sample_every = 7
    frames_to_predict = 12
    sampled_frames = 0

    if os.path.exists('./sampled_frames'):
        shutil.rmtree('./sampled_frames')
    os.mkdir('./sampled_frames')
    prediction = None
    texto = ""
    #Definir el texto y las propiedades del texto
    font = cv2.FONT_HERSHEY_SIMPLEX
    escala = 1
    color = (255, 255, 255)  # Color en formato BGR
    grosor = 2
    tipo_linea = cv2.LINE_AA
    capturing = False
    while True:
        # Lee el frame desde la cámara
        ret, frame = cap.read()

        # Verifica si la lectura fue exitosa
        if not ret:
            print("Error al leer el frame de la cámara.")
            break

        if frame_number % sample_every == 0 and prediction is None:
            # Guarda el frame en un archivo
            cv2.imwrite('./sampled_frames/frame_'+str(frame_number)+'.jpg', frame)
            sampled_frames += 1
            capturing = True
        elif frames_to_predict == sampled_frames:
            capturing = False
            if prediction is None:
                if os.path.exists('./tmp'):
                    shutil.rmtree('./tmp')
                shutil.copytree('./sampled_frames', './tmp')
                prediction = FramesPredictor()
                prediction.start()
                shutil.rmtree('./sampled_frames')
                os.mkdir('./sampled_frames') 
                sampled_frames = 0
        if prediction and prediction.result is not None:
            # Show text
            print(prediction.result)
            best_prediction = None
            best_prediction_count = 0
            for key in prediction.result.keys():
                if best_prediction is None:
                    best_prediction = key
                    best_prediction_count = prediction.result[key]
                elif prediction.result[key] > best_prediction_count:
                    best_prediction = key
                    best_prediction_count = prediction.result[key]
            # Añadir el texto al frame
            texto = best_prediction
            prediction = None
            shutil.rmtree('./tmp')
        elif prediction is not None:
            capturing = False
            cv2.putText(frame, "Procesando", (50, 100), font, escala, (0,0,255), grosor, tipo_linea)
        frame_number += 1
        # Rompe el bucle si se presiona la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        cv2.putText(frame, texto, (250, 400), font, escala, color, grosor, tipo_linea)
        if capturing:
            cv2.putText(frame, "Capturando", (50, 100), font, escala, (0,0,255), grosor, tipo_linea)
        # Muestra el frame en una ventana
        cv2.imshow('frame', frame)

    # Libera la captura y cierra la ventana
    cap.release()
    cv2.destroyAllWindows()

start_live_demo()


{'hola': 2, 'donde': 3, 'disculpa': 5, 'como_estas': 2}
{'hola': 8, 'vives': 3, 'donde': 1}
